In [1]:
%load_ext autoreload
%autoreload 2

import os

import ipdb
import matplotlib
from tqdm import tqdm

from torch import nn

from utils.config import opt
from data.dataset import Dataset, TestDataset, inverse_normalize
from data.Data import COCODataset
from model import FasterRCNNVGG16, FasterRCNNVGG16Extractor
from faster_rcnn_lstm import FasterRCNN_LSTM
from torch.utils import data as data_
from trainer import FasterRCNNTrainer,LSTMTrainer
from utils import array_tool as at
from utils.vis_tool import visdom_bbox
from utils.eval_tool import eval_detection_voc
from model.predictor import PredictNet
import pandas as pd
import numpy as np

C:\Users\Administrator\Desktop\rcnn lstm\model\utils\nms\non_maximum_suppression.py:12: UserWarning: 
    the python code for non_maximum_suppression is about 2x slow
    It is strongly recommended to build cython code: 
    `cd model/utils/nms/; python3 build.py build_ext --inplace
  `cd model/utils/nms/; python3 build.py build_ext --inplace''')


In [2]:
embedding_file = 'G:\\data\\coco_filtered_word2vec_1024'
dataset = COCODataset(embedding_file,opt,True)
opt.n_class =dataset.n_class
faster_rcnn = FasterRCNNVGG16()
print(list(faster_rcnn.extractor.parameters()))

loading annotations into memory...
Done (t=2.04s)
creating index...
index created!
loading annotations into memory...
Done (t=24.54s)
creating index...
index created!
loading annotations into memory...
Done (t=0.14s)
creating index...
index created!
loading annotations into memory...
Done (t=2.10s)
creating index...
index created!
len model.feature 44
[Parameter containing:
tensor([[[[ 8.2833e-02,  2.7968e-02,  7.7096e-02],
          [ 4.9341e-02, -3.3441e-02,  1.9572e-02],
          [ 8.0300e-02,  7.7076e-02,  8.3349e-02]],

         [[-4.4296e-02, -1.7748e-01, -4.8706e-02],
          [-1.1003e-01, -2.7530e-01, -1.3474e-01],
          [-5.9982e-03, -6.1375e-02,  1.6822e-02]],

         [[ 2.7480e-02, -6.6769e-02,  4.3955e-02],
          [-2.6662e-02, -1.4995e-01, -3.3615e-02],
          [ 5.2778e-02,  1.7143e-02,  8.6744e-02]]],


        [[[-1.2628e-02,  3.0218e-02, -2.6930e-02],
          [-1.3764e-02,  1.1993e-01, -6.6263e-03],
          [-2.6019e-02, -8.3535e-03, -3.9197e-02]],

 

        -6.2843e-01, -4.9615e-01], requires_grad=True)]


In [3]:
trainer = FasterRCNNTrainer(faster_rcnn).cuda()
opt.caffe_pretrain = False

Exception in user code:
------------------------------------------------------------


Traceback (most recent call last):
  File "G:\Anaconda3\lib\site-packages\urllib3\connection.py", line 159, in _new_conn
    (self._dns_host, self.port), self.timeout, **extra_kw)
  File "G:\Anaconda3\lib\site-packages\urllib3\util\connection.py", line 80, in create_connection
    raise err
  File "G:\Anaconda3\lib\site-packages\urllib3\util\connection.py", line 70, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [WinError 10061] 由于目标计算机积极拒绝，无法连接。

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "G:\Anaconda3\lib\site-packages\urllib3\connectionpool.py", line 600, in urlopen
    chunked=chunked)
  File "G:\Anaconda3\lib\site-packages\urllib3\connectionpool.py", line 354, in _make_request
    conn.request(method, url, **httplib_request_kw)
  File "G:\Anaconda3\lib\http\client.py", line 1229, in request
    self._send_request(method, url, body, headers, encode_chunked)
  File "G:\Anaconda3\lib\http\client.py

In [4]:
lstm = nn.LSTM(input_size=dataset.word_embedding.vector_size, hidden_size=opt.hidden_size, batch_first=True)
predict_param = [lstm.hidden_size, opt.n_class]
text_predictor = PredictNet(predict_param)
text_predictor.parameters()
lstm_trainer = LSTMTrainer(lstm,text_predictor).cuda()
print('load data')
dataloader = data_.DataLoader(dataset, \
                              batch_size=1, \
                              shuffle=True, \
                              # pin_memory=True,
                              num_workers=opt.num_workers)

load data


In [5]:
for epoch in range(opt.epoch):
    for ii, (img, bbox_,text, label_, scale) in tqdm(enumerate(dataloader)):
        # train faster rcnn
        scale = at.scalar(scale)
        img, bbox,text, label = img.cuda().float(), bbox_.cuda(),text.cuda(), label_.cuda()
        trainer.train_step(img, bbox, label, scale)
        # train lstm
        lstm_trainer.lstm_step(text, label)

0it [00:00, ?it/s]

org image [[[197. 196. 195. ...  35.  36.  35.]
  [193. 194. 194. ...  37.  36.  34.]
  [195. 195. 196. ...  38.  37.  35.]
  ...
  [ 81.  88.  89. ...  62.  76.  82.]
  [ 85.  81.  81. ...  57.  68.  74.]
  [ 80.  80.  82. ...  52.  65.  71.]]

 [[200. 198. 196. ...  38.  39.  39.]
  [196. 195. 195. ...  40.  39.  38.]
  [196. 196. 197. ...  41.  40.  39.]
  ...
  [ 80.  87.  88. ...  69.  83.  88.]
  [ 82.  78.  80. ...  61.  75.  81.]
  [ 77.  77.  79. ...  57.  70.  76.]]

 [[215. 213. 214. ...  45.  48.  48.]
  [213. 213. 213. ...  49.  48.  47.]
  [214. 214. 215. ...  50.  49.  48.]
  ...
  [ 86.  93.  94. ...  79.  93. 100.]
  [ 89.  85.  86. ...  70.  83.  89.]
  [ 84.  84.  86. ...  61.  76.  82.]]]
img [[[ 1.2440403   1.238881    1.2295187  ... -1.504815   -1.509887
   -1.5181816 ]
  [ 1.2171326   1.2181113   1.2170435  ... -1.5007674  -1.5130225
   -1.5243859 ]
  [ 1.1985146   1.2037402   1.2104136  ... -1.4918514  -1.511445
   -1.525944  ]
  ...
  [-0.6889029  -0.7114971  -

1it [00:04,  4.45s/it]

org image [[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]]
img [[[-2.117904  -2.117904  -2.117904  ... -2.117904  -2.117904  -2.117904 ]
  [-2.117904  -2.117904  -2.117904  ... -2.117904  -2.117904  -2.117904 ]
  [-2.117904  -2.117904  -2.117904  ... -2.117904  -2.117904  -2.117904 ]
  ...
  [-2.117904  -2.117904  -2.117904  ... -2.117904  -2.117904  -2.117904 ]
  [-2.117904  -2.117904  -2.117904  ... -2.117904  -2.117904  -2.117904 ]
  [-2.117904  -2.117904  -2.117904  ... -2.117904  -2.117904  -2.117904 ]]

 [[-2.0357141 -2.0357141 -2.0357141 ... -2

RuntimeError: CUDA out of memory. Tried to allocate 88.00 MiB (GPU 0; 4.00 GiB total capacity; 1.71 GiB already allocated; 7.98 MiB free; 56.88 MiB cached)

In [ ]:
print(list(faster_rcnn.extractor.parameters()))

In [ ]:
import torch as t

In [ ]:
t.load('G:\\models\\fasterrcnn_torchvision_pretrain.pth')

In [ ]:
dic[3]